<a href="https://colab.research.google.com/github/rahiakela/StatQuest-Neural-Networks-and-AI/blob/main/chapter_11/chapter_11_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The StatQuest Illustrated Guide to Neural Networks and AI
## Chapter 11 - Attention

Copyright 2024, Joshua Starmer

-----

This tutorial is from the book, **[The StatQuest Illustrated Guide to Neural Networks and AI](https://www.amazon.com/dp/B0DRS71QVQ)**.

In this notebook, we will build and train a Seq2Seq/Encoder-Deocder model Attention. Compared with what we did in Chapter 10, the Encoder and Decoder in this example will be a little simplier, although it doesn't technically need to be - we can make it as fancy as we want.  Anyway, this model only uses a single layer of LSTMs and no stacking, as seen in the picture below.

<img src="https://github.com/StatQuest/signa/blob/main/chapter_11/images/full_model.png?raw=1" alt="an encoder-decoder model with attention" style="width: 800px;">

In this tutorial, you will...

**NOTE:**

This tutorial assumes that you have read through the chapter on **Seq2Seq and Encoder-Decoder Models** and the chapter on **Attention** in **The StatQuest Illustrated Guide to Neural Networks and AI**.

----

# Import the modules that will do all the work

The very first thing we need to do is load a bunch of Python modules. Python itself is just a basic programming language. These modules give us extra functionality to create and train a Neural Network.

In [1]:
%%capture
# %%capture prevents this cell from printing a ton of STDERR stuff to the screen

## First, check to see if lightning is installed, if not, install it.
##
## NOTE: If you **do** need to install something, just know that you may need to
##       restart your session for python to find the new module(s).
##
##       To restart your session:
##       - In Google Colab, click on the "Runtime" menu and select
##         "Restart Session" from the pulldown menu
##       - In a local jupyter notebook, click on the "Kernel" menu and select
##         "Restart Kernel" from the pulldown menu
import pip
try:
  __import__("lightning")
except ImportError:
  pip.main(['install', "lightning"])

In [1]:
import torch ## torch let's us create tensors and also provides helper functions
import torch.nn as nn ## torch.nn gives us nn.Module(), nn.Embedding() and nn.Linear()
import torch.nn.functional as F # This gives us the softmax() and argmax()
from torch.optim import Adam ## We will use the Adam optimizer, which is, essentially,
                             ## a slightly less stochastic version of stochastic gradient descent.
from torch.utils.data import TensorDataset, DataLoader ## We'll store our data in DataLoaders

import lightning as L ## Lightning makes it easier to write, optimize and scale our code

## NOTE: If you get an error running this block of code, it is probably
##       because you installed a new package earlier and forgot to
##       restart your session for python to find the new module(s).
##
##       To restart your session:
##       - In Google Colab, click on the "Runtime" menu and select
##         "Restart Session" from the pulldown menu
##       - In a local jupyter notebook, click on the "Kernel" menu and select
##         "Restart Kernel" from the pulldown menu

----

# Create the datasets that we will use for training Encoder-Decoder model

To make the model at least a little bit interesting, we will translate two english phrases, **Let's go** and **to go** into spanish. **Let's go** should translate to **vamos \<EOS\>** and **to go** should translate to **ir \<EOS\>**.

In [2]:
## first, we create a dictionary that maps vocabulary tokens to id numbers...
english_token_to_id = {'lets': 0,
                       'to': 1,
                       'go': 2,
                       '<EOS>': 3 ## <EOS> = end of sequence
                      }
## ...then we create a dictionary that maps the ids to tokens. This will help us interpret the output.
## We use the "map()" function to apply the "reversed()" function to each tuple (i.e. ('lets', 0)) stored
## in the token_to_id dictionary. We then use dict() to make a new dictionary from the
## reversed tuples.
english_id_to_token = dict(map(reversed, english_token_to_id.items()))

spanish_token_to_id = {'ir': 0,
                       'vamos': 1,
                       'y': 2,
                       '<EOS>': 3}
spanish_id_to_token = dict(map(reversed, spanish_token_to_id.items()))

inputs = torch.tensor([[english_token_to_id["lets"],
                        english_token_to_id["go"]],

                       [english_token_to_id["to"],
                        english_token_to_id["go"]]])

labels = torch.tensor([[spanish_token_to_id["vamos"],
                        spanish_token_to_id["<EOS>"]],

                       [spanish_token_to_id["ir"],
                        spanish_token_to_id["<EOS>"]]])

Now that we have created the data that we want to train the embeddings with, we'll store it in a `DataLoader`. Since our dataset is so small, using a `DataLoader` is a little bit of an overkill, but it it's easy to do, and it will allow us to easily scale up to a much larger vocabulary when the time comes.

In [3]:
## Now let's package everything up into a DataLoader...
dataset = TensorDataset(inputs, labels)
dataloader = DataLoader(dataset)

----

# Build and Train a Seq2Seq/Encoder-Decoder Model with Attention from Scratch

In [4]:
class EncoderDecoderAttention(L.LightningModule):

    def __init__(self, max_len=2):

        super().__init__()

        self.max_decoder_length = max_len

        L.seed_everything(seed=42)

        #################################
        ##
        ## ENCODING
        ##
        #################################
        self.encoder_embeddings = nn.Embedding(num_embeddings=4, # num_embeddings = # of tokens in input vocabulary
                                       embedding_dim=1)  # embedding_dim = # of embedding values per token

        self.encoder_lstm = nn.LSTM(input_size=1, # encoder_size = number of inputs
                                    hidden_size=1,# hidden_size = number of outputs
                                    num_layers=1) # num_layers = how many lstm's to stack
                                                  #              If there are 2 layers, then the short term memory from the
                                                  #              first layer is used as input to the second layer

        #################################
        ##
        ## DECODING
        ##
        #################################
        self.decoder_embeddings = nn.Embedding(num_embeddings=4,
                                       embedding_dim=1)

        self.decoder_lstm = nn.LSTM(input_size=1,
                                    hidden_size=1,
                                    num_layers=1)

        self.decoder_fc = nn.Linear(in_features=2,  # in_features = number of outputs per lstm * 2 (encoder + decoder)
                                   out_features=4) # out_features = number of words in the output vocabulary

        self.loss = nn.CrossEntropyLoss()


    def forward(self, input, output=None):

        # outputs = torch.zeros(self.max_decoder_length, 4)

        #################################
        ##
        ## ENCODING
        ##
        #################################
        ## first, use the encoder stage to create an intermediate encoding of the input text
        encoder_embeddings = self.encoder_embeddings(input)
        encoder_lstm_output, (encoder_lstm_hidden, encoder_lstm_cell) = self.encoder_lstm(encoder_embeddings)

        #################################
        ##
        ## DECODING
        ##
        #################################
        ## We start by initializing the decoder with the <EOS> token...
        decoder_token_id = torch.tensor([spanish_token_to_id["<EOS>"]])
        decoder_embeddings = self.decoder_embeddings(decoder_token_id)
        decoder_lstm_output, (decoder_lstm_hidden, decoder_lstm_cell) = self.decoder_lstm(decoder_embeddings,
                                                                                          (encoder_lstm_hidden,
                                                                                           encoder_lstm_cell))

        ## Calculate attention here (using unnormalized cosine similarity, aka, dot product, to score the alignment)
        ## In other words, we're trying to find words in the input that are used in the same context as those proposed
        ## by the lstm.
        sims = torch.matmul(decoder_lstm_output, encoder_lstm_output.transpose(dim0=0, dim1=1))
        ## Apply softmax to determine what percent of each token's value to
        ## use in the final attention values.
        attention_percents = F.softmax(sims, dim=1)

        ## Scale the values by their associated percentages and add them up.
        attention_values = torch.matmul(attention_percents, encoder_lstm_output)

        ## lastly, we need to concatenate the attention values with the short term
        ## memories from the first decoder lstm
        values_to_fc_layer = torch.cat((attention_values, decoder_lstm_output), 1)

        output_values = self.decoder_fc(values_to_fc_layer)
        outputs = output_values
        predicted_id = torch.tensor([torch.argmax(output_values)])
        predicted_ids = predicted_id

        for i in range(1, self.max_decoder_length):
            if (output == None): # using the model...
                if (predicted_id == spanish_token_to_id["<EOS>"]): # if the prediction is <EOS>, then we are done
                    break
                decoder_embeddings = self.decoder_embeddings(predicted_id)
            else:
                decoder_embeddings = self.decoder_embeddings(torch.tensor([output[i-1]]))

            decoder_lstm_output, (decoder_lstm_hidden, decoder_lstm_cell) = self.decoder_lstm(decoder_embeddings,
                                                                                              (decoder_lstm_hidden,
                                                                                               decoder_lstm_cell))
            sims = torch.matmul(decoder_lstm_output, encoder_lstm_output.transpose(dim0=0, dim1=1))
            ## Apply softmax to determine what percent of each token's value to
            ## use in the final attention values.
            attention_percents = F.softmax(sims, dim=1)

            ## Scale the values by their associated percentages and add them up.
            attention_values = torch.matmul(attention_percents, encoder_lstm_output)

            values_to_fc_layer = torch.cat((attention_values, decoder_lstm_output), 1)

            output_values = self.decoder_fc(values_to_fc_layer)
            # outputs[i] = output_values
            outputs = torch. cat((outputs, output_values), 0)
            predicted_id = torch.tensor([torch.argmax(output_values)])
            predicted_ids = torch.cat((predicted_ids, predicted_id))

        return(outputs)


    def configure_optimizers(self): # this configures the optimizer we want to use for backpropagation.
        return Adam(self.parameters(), lr=0.1) ## NOTE: Setting the learning rate to 0.1 trains way faster than
                                               ## using the default learning rate, lr=0.001


    def training_step(self, batch, batch_idx): # take a step during gradient descent.
        input_tokens, labels = batch # collect input
        output = self.forward(input_tokens[0], labels[0]) # run input through the neural network
        loss = self.loss(output, labels[0]) ## self.loss = cross entropy
        ###################
        ##
        ## Logging the loss
        ##
        ###################
        # self.log("train_loss", loss)

        return loss

Now that we have created the `seq2seq_attention()` class, let's just run the phrase **Let's go** through it to see what it gets translated into.

In [5]:
model = EncoderDecoderAttention()
outputs = model.forward(input=torch.tensor([english_token_to_id["lets"],
                                            english_token_to_id["go"]]), ## translate "lets go", we should get "vamos <EOS>"
                        output=None)

print("Translated text:")
predicted_ids = torch.argmax(outputs, dim=1)
for id in predicted_ids:
    print("\t", spanish_id_to_token[id.item()])

INFO: Seed set to 42
INFO:lightning.fabric.utilities.seed:Seed set to 42


Translated text:
	 y
	 y


And we see that **Let's go** was translated to **y y** instead of what we wanted, which was **vamos \<EOS\>**. So let's train the model!

In [6]:
trainer = L.Trainer(max_epochs=20, accelerator="cpu")
trainer.fit(model, train_dataloaders=dataloader)

INFO: Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
INFO:lightning.pytorch.utilities.rank_zero:Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: 
  | Name               | Type             | Params | Mode 
----------------------------------------------------------------
0 | encoder_embeddings | Embedding        | 4      | train
1 | encoder_lstm       | LSTM             | 16     | train
2 | decoder_embed

Training: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=20` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.


Now let's see if the model correctly translates **Let's go** into **vamos \<EOS\>**...

In [7]:
outputs = model.forward(input=torch.tensor([english_token_to_id["lets"],
                                            english_token_to_id["go"]]), ## translate "lets go", we should get "vamos <EOS>"
                        output=None)

print("Translated text:")
predicted_ids = torch.argmax(outputs, dim=1)
for id in predicted_ids:
    print("\t", spanish_id_to_token[id.item()])

Translated text:
	 vamos
	 <EOS>


...and it does!

**BAM!**

Now let's see if the model correctly translates **to go** to **ir \<EOS\>**...

In [8]:
outputs = model.forward(input=torch.tensor([english_token_to_id["to"],
                                            english_token_to_id["go"]]), ## translate "to go", we should get "ir <EOS>"
                        output=None)

print("Translated text:")
predicted_ids = torch.argmax(outputs, dim=1)
for id in predicted_ids:
    print("\t", spanish_id_to_token[id.item()])

Translated text:
	 ir
	 <EOS>


...and it does!

**DOUBLE BAM!!**

----

# Print Out Weights and Biases

Relatively early in **Chapter 11** we had a note saying that if you wanted to do the math by hand, you'd have to print out the weights and biases when we code it. So, here are the weights and biases...

In [9]:
print("After optimization, the parameters are...")
for name, param in model.named_parameters():
    print(name, param.data)

After optimization, the parameters are...
encoder_embeddings.weight tensor([[-1.7787],
        [ 2.0923],
        [-0.1636],
        [ 0.2303]])
encoder_lstm.weight_ih_l0 tensor([[-1.6984],
        [-2.2462],
        [ 1.4013],
        [-1.5993]])
encoder_lstm.weight_hh_l0 tensor([[-0.7744],
        [-1.1266],
        [ 1.5920],
        [-0.4829]])
encoder_lstm.bias_ih_l0 tensor([ 1.8437,  1.4772, -1.0555, -0.0629])
encoder_lstm.bias_hh_l0 tensor([ 0.6121,  1.2121, -0.9948,  0.3456])
decoder_embeddings.weight tensor([[-0.8066],
        [-2.2045],
        [-0.9890],
        [ 2.8455]])
decoder_lstm.weight_ih_l0 tensor([[-0.4647],
        [ 2.1268],
        [-1.5542],
        [ 0.4553]])
decoder_lstm.weight_hh_l0 tensor([[-1.3270],
        [ 1.8276],
        [-0.5032],
        [-1.9216]])
decoder_lstm.bias_ih_l0 tensor([ 1.8560, -0.7807,  0.7111,  3.0293])
decoder_lstm.bias_hh_l0 tensor([ 2.1653, -0.9467,  0.8675,  2.3154])
decoder_fc.weight tensor([[ 0.4913, -1.7664],
        [-1.1043, 

**TRIPLE BAM!!!**